<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2020</h1>
<h1 style="font-size: 250%;">Problem Set #2 </h1>
<h1>Issued Tuesday, 2/5/2020; Due Tuesday, 2/18/2020</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of 70.71% for up to two class meetings after the due date. Feel free to get help from others, but **the work you submit in should be your own.**

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [5]:
# Replace the following string values with the requested information
class Student:
    first = "Rachel"
    last = "Tucker"
    onyen = "rach1234"
    pid = "730155154"

You will need a the following sequence collection of gene promoter regions in which you will search for <a href="http://csbio.unc.edu/mcmillan/Comp55520/data/motifs.fa" download="motifs.txt">transcription binding factor motifs</a>.

The cell below provides all functions and imports necessary for this problem set. Do not import any addtional packages. Also, make sure that you ***run the following cell***.

In [19]:
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into two lists. One of 
        headers and a second list of sequences.
        The ith index of each list correspond."""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def MedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

---
**Problem #1:** In the cell below use the given *MedianStringMotifSearch()*, and *ScanAndScoreMotif()* functions to find an optimal 8-base motif pattern and its location in each of the 16 given promoter regions in <a href="http://csbio.unc.edu/mcmillan/Comp555S20/data/motifs.fa" download="motifs.txt">motifs.fa</a>. Based on this result, make a prediction of how long it would have taken to find an optimal 10-base motif.

In [ ]:
# Read in the motifs.fa FASTA file and use its sequence 
# list as "dna" and run the given code fragment.

headers, dna =  loadFasta("data/motifs.fa")







%time MedianStringMotifSearch(dna,8) 

---
**Problem #2:** A simple optimization can be applied to *MedianStringMotifSearch()* as follows: If ever during the *ScanAndScoreMotif()* function the Hamming distance (i.e. *totalDist* in *ScanAndScoreMotif()*) exceeds the smallest Hamming distance seen thus far (*minHammingDist* in *MedianStringMotifSearch()*), the scanning through sequences can be terminated early. Implement this strategy and use it to search for the best 10-base motif. (Note: this requires that you do one at least of the following, add an additional argument to the *ScanAndScoreMotif()*, create a global variable shared by both *BetterMedianStringMotifSearch()* and *ScanAndScoreMotif()*, or encapuslate *ScanAndScoreMotif()* as an inner function of *BetterMedianStringMotifSearch()*). Run your new optimized version and report its run time. This optimization is technically not a branch-and-bound strategy; explain why?

In [ ]:
# Modify the MedianStringMotifSearch(), and ScanAndScoreMotif() 
# functions here and run them using the given code fragment.

def BetterScanAndScoreMotif(DNA, motif, minHam):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist

                
        bestAlignment.append(bestS)
        totalDist += minHammingDist
        if(totalDist > minHam):
            break
    return bestAlignment, totalDist

def BetterMedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = BetterScanAndScoreMotif(DNA, motif, minHammingDist)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

headers, dna =  loadFasta("data/motifs.fa")

%time BetterMedianStringMotifSearch(dna,10)

---
**Problem #3:** The *ContainedMotifSearch()* algortihm from lecture 5 can be used to find a heuristic solution for the motif search problem considerably faster than the *MedianStringMotifSearch()*. However, this solution may not be best overall solution. *ContainedMotifSearch()* can still be used to speed up *MedianStringMotifSearch()* using the following approach: First, use ContainedMotifSearch() to establish an initial *bestAlignment* and an upper-bound for the ultimate Hamming distance. Then, use it as the intitial setting of the minimum Hamming distance used in *MedianStringMotifSearch()* in conjuction with your modified version of *ScanAndScoreMotif()* from Problem #2. 

In the cell provided below write *EvenBetterMedianMotifSearch()*.

In [17]:
# THIS CELL WILL BE GRADED
# This cell should include *ALL* functions called by EvenBetterMedianMotifSearch(),
# such as ContainedMedianSearch() and ScanAndScoreMotif(). Any test or 
# validation code should be placed in another cell 

def BetterScanAndScoreMotif(DNA, motif, minHam):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist

                
        bestAlignment.append(bestS)
        totalDist += minHammingDist
        if(totalDist > minHam):
            break
    return bestAlignment, totalDist

def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = BetterScanAndScoreMotif(DNA, motif, minHammingDist)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

def EvenBetterMedianMotifSearch(DNA,k):
    bestAlignment, minHammingDist, kmer = ContainedMotifSearch(DNA, k) 
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = BetterScanAndScoreMotif(DNA, motif, minHammingDist)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

The following cell below is provided for testing your code. You should test it on <a href="http://csbio.unc.edu/mcmillan/Comp555S20/data/motifs.fa" download="motifs.txt">motifs.fa</a>. This cell will not be considered during grading.

In [18]:
headers, dna =  loadFasta("data/motifs.fa")

%time EvenBetterMedianMotifSearch(dna,10)

3861 Motifs in our set


KeyboardInterrupt: 

**Problem #4:** Next consider how the prefixes of candidate motifs can be used to determine if further extensions might possibly beat the *ContainedMotifSearch()* solution. This last change makes the algorithm truly "branch-and-bound" and requires that succesive *patterns* be generated using a different approach than the itertools product used in *MedianStringMotifSearch()*.

In the cell provided below write *BestMedianMotifSearch()*.

In [22]:
# THIS CELL WILL BE GRADED
# This cell should include *ALL* functions called by BestMedianMotifSearch(),
# such as ContainedMedianSearch() and ScanAndScoreMotif(). Any test or 
# validation code should be placed in another cell 
def BetterScanAndScoreMotif(DNA, motif, minHam):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist

                
        bestAlignment.append(bestS)
        totalDist += minHammingDist
        if(totalDist > minHam):
            break
    return bestAlignment, totalDist

def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = BetterScanAndScoreMotif(DNA, motif, minHammingDist)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer, motifSet

def BestMedianMotifSearch(DNA,k):
    bestAlignment, minHammingDist, kmer, motifSet = ContainedMotifSearch(DNA, k) 
    for pattern in motifSet:
        motif = ''.join(pattern)
        align, dist = BetterScanAndScoreMotif(DNA, motif, minHammingDist)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

In [26]:
# This cell is provided for testing your answer to problem 4. It will not be graded.
headers, dna =  loadFasta("data/motifs.fa")

%time BestMedianMotifSearch(dna,10)

3861 Motifs in our set
Wall time: 1min 20s


([115, 180, 152, 235, 22, 66, 84, 78, 61, 39, 196, 98, 111, 14, 147, 35],
 40,
 'CATGCCGACT')

**Problem #5:** Recall that *ContainedMotifSearch()* considers only k-mers that appear in the given sequences as possible motifs. Modify *ContainedMotifSearch()* so that it considers all k-mers that have a Hamming distance of 1 or less from the set of k-mers that appear in the given sequences. Call your new search algorithm *DistanceOneContainedMotifSearch()*.

In [25]:
# THIS CELL WILL BE GRADED.
# This cell should include *ALL* functions called by DistanceOneContainedMotifSearch().
# All test and validation code should be placed in another cell


def BetterScanAndScoreMotif(DNA, motif, minHam):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist

                
        bestAlignment.append(bestS)
        totalDist += minHammingDist
        if(totalDist > minHam):
            break
    return bestAlignment, totalDist

def DistanceOneContainedMotifSearch(DNA,k):
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    
    motifSet = set()
    motifSet2 = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    
    for motif in motifSet:
        q = 0
        for i in range(len(motif)):
            
            motif1 = list(motif)
            motif1[q] = 'A'
            str1 = ''.join(motif1)
            motifSet2.add(str1)
            
            motif2 = list(motif)
            motif2[q] = 'C'
            str2 = ''.join(motif2)
            motifSet2.add(str2)
            
            motif3 = list(motif)
            motif3[q] = 'G'
            str3 = ''.join(motif3)
            motifSet2.add(str3)
            
            motif4 = list(motif)
            motif4[q] = 'T'
            str4 = ''.join(motif4)
            motifSet2.add(str4)
            
            q = q + 1
            
    for motif in motifSet2:
        bestAlignment, totalDist = BetterScanAndScoreMotif(DNA, motif, minHammingDist)
        
        if (totalDist <= 1):
            if(kmer != ''):
                bestAlignment = [s for s in align]
                minHammingDist = dist
                kmer = motif
    return bestAlignment, minHammingDist, kmer

In [26]:
# This cell is provided for testing your answer to problem 5. It will not be graded.

headers, dna =  loadFasta("data/motifs.fa")

%time DistanceOneContainedMotifSearch(dna,10)

3861 Motifs in our set


KeyboardInterrupt: 

---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.